# Working with Cloud (Remote) Back-ends
## So far
- We've learnt to create a synthetic dataset
- Train a logical function to make predictions from the synthetic dataset
- Test and develop an API server with Flask
- Deploy our prediction function to Heroku

## In this section
We will learn to interact with Google's Firebse and create a document collection **to demonstrate the usecase when our application needs to collect and store user data**.

## Setup and Authentication

In [1]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore

#### Point to Firebase credentials

In [2]:
cred = credentials.Certificate("../configs/app_creds.json")

#### Initialize firebase client as admin

In [3]:
firebase_admin.initialize_app(cred,
    {
        "databaseURL": "https://save-nyc-demo.firebaseio.com/"
})

## Interfacing with your Firebase Collections

#### Load data to bootstrap

In [4]:
df = pd.read_csv("../data/superhero_timesheets.csv")
df

,superhero,date,active_on_duty
0,Daredevil,2023-07-17,0
1,Deadpool,2023-07-18,1
2,Dr X,2023-07-17,0
3,Dr X,2023-07-18,1
4,Luke Cage,2023-07-16,1
5,Luke Cage,2023-07-18,0
6,Mr Fantastic,2023-07-16,1
7,Mr Fantastic,2023-07-18,1
8,Peeping Tom,2023-07-16,1
9,Peeping Tom,2023-07-17,1


#### Connect to your desired collection

In [5]:
db = firestore.client()
doc_ref = db.collection("superhero_timesheets")

#### Bootstrap all records

In [6]:
%%time
tmp = df.to_dict(orient="records")

list(map(lambda x: doc_ref.add(x), tmp))

CPU times: user 97.9 ms, sys: 70.5 ms, total: 168 ms
Wall time: 6.66 s


[(DatetimeWithNanoseconds(2023, 7, 18, 13, 50, 9, 408997, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x1179d9e80>),
 (DatetimeWithNanoseconds(2023, 7, 18, 13, 50, 9, 731504, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x1179d9970>),
 (DatetimeWithNanoseconds(2023, 7, 18, 13, 50, 10, 265185, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x1179d9cd0>),
 (DatetimeWithNanoseconds(2023, 7, 18, 13, 50, 10, 504890, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x117950100>),
 (DatetimeWithNanoseconds(2023, 7, 18, 13, 50, 11, 24346, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x117a00af0>),
 (DatetimeWithNanoseconds(2023, 7, 18, 13, 50, 11, 261725, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x1179eb520>),
 (DatetimeWithNanos

#### Query to fetch all records for today

In [7]:
today = pd.Timestamp.now().date()
res_0 = pd.DataFrame([
    doc.to_dict() for doc in db.collection(u"superhero_timesheets").where(u"date", u">=", str(today)).stream()
])

res_0

,active_on_duty,date,superhero
0,0,2023-07-18,The Comedian
1,1,2023-07-18,Mr Fantastic
2,1,2023-07-18,Dr X
3,0,2023-07-18,Spiderman
4,1,2023-07-18,Deadpool
5,0,2023-07-18,Luke Cage


#### Adding a record

In [8]:
%%time
doc_ref.add({
    "superhero": "Leonardo",
    "active_on_duty": 1,
    "date": "2022-05-31"
})

CPU times: user 1.86 ms, sys: 2.41 ms, total: 4.27 ms
Wall time: 196 ms


(DatetimeWithNanoseconds(2023, 7, 18, 13, 50, 31, 950705, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x1179d9790>)

In [9]:
res_1 = pd.DataFrame([
    doc.to_dict() for doc in db.collection(u"superhero_timesheets").where(u"date", u">=", str(today)).stream()
])

res_1

,superhero,date,active_on_duty
0,Spiderman,2022-05-31,1
1,Deadpool,2022-05-31,1
2,Leonardo,2022-05-31,1
3,Mr Fantastic,2022-05-31,1
4,Luke Cage,2022-05-31,0
5,Daredevil,2022-05-31,1
6,Wolverine,2022-05-31,1
7,Peeping Tom,2022-05-31,1


#### Dropping a collection

In [9]:
%%time
def delete_collection(coll_ref, batch_size=200):
    docs = coll_ref.limit(batch_size).stream()
    deleted = 0

    for doc in docs:
        print(f'Deleting doc {doc.id} => {doc.to_dict()}')
        doc.reference.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)

delete_collection(doc_ref)

Deleting doc 0YpyPk8SnQvEO8R1xvKB => {'date': '2022-05-29', 'active_on_duty': 0, 'superhero': 'Dr X'}
Deleting doc 1q0YCAyFmhKYABwpQZSy => {'date': '2022-05-31', 'active_on_duty': 1, 'superhero': 'Daredevil'}
Deleting doc 2X0vxiXIB7ZX65qMbW7s => {'active_on_duty': 0, 'date': '2023-07-18', 'superhero': 'The Comedian'}
Deleting doc 4Dqg5rFUgn3fXp6RwPer => {'active_on_duty': 1, 'date': '2022-05-30', 'superhero': 'Wolverine'}
Deleting doc 7O8mMSYbLKEY1Bk82aWD => {'active_on_duty': 0, 'date': '2022-05-30', 'superhero': 'Spiderman'}
Deleting doc 7iIPjxrRHFhUo1cBI883 => {'active_on_duty': 1, 'date': '2023-07-18', 'superhero': 'Mr Fantastic'}
Deleting doc A9h22aRh7zumjY6cnW4f => {'active_on_duty': 1, 'date': '2022-05-31', 'superhero': 'Deadpool'}
Deleting doc Bp7OINziADOW7PpuUMg5 => {'active_on_duty': 1, 'date': '2023-07-16', 'superhero': 'Spiderman'}
Deleting doc EtV6RvJguUi7sGvRdz4l => {'active_on_duty': 0, 'date': '2022-05-31', 'superhero': 'Peeping Tom'}
Deleting doc KmuFQovkAPhjvwTglPyJ =

## Outro
Thank you for following this workshop all the way to the ends. All credits to [Firebase Docs](https://firebase.google.com/docs) for code snippets used in this notebook.